# IMPORTS

In [1]:
import yaml
import numpy as np
import firebase_admin
import pandas as pd

import datetime

In [2]:
from oauth2client import file,client, tools

In [3]:
#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

In [4]:
#GSHEETs 
# code labe for bigginers - https://codelabs.developers.google.com/codelabs/gsuite-apis-intro/#7
import httplib2
import os, io
import argparse
import auth

import unidecode

from httplib2 import Http

from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials

# CONNECT TO FIREBASE

In [5]:
filename='##'

In [6]:
# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## TEST

In [7]:
col_query = db.collection('Accounting').where(u'project', u'==', u'opunMdqGyXDlfQ9DYIbN')

In [8]:
cuentas_auara = col_query.stream()

In [10]:
for acc in cuentas_auara:
    print(u'{} => {}'.format(acc.id, acc.to_dict()))

## ABRIR VENTAS

In [14]:
ventas = db.collection(
                       'Nikarit_Sales'
                      ).where(u'origin', u'==', u'Shopify').stream()

In [16]:
l_ventas = []
for venta in ventas:
    d_venta = venta.to_dict()
    if d_venta['creation_date'][0:4]=='2020':
        l_ventas.append(d_venta)

In [35]:
datetime.datetime.strptime(l_ventas[0])

datetime.datetime(2020, 4, 28, 9, 12, 24)

In [72]:
l_ventas3 = []
for venta in l_ventas:
    #2020-05-08T13:31:15+02:00
    creation_date = datetime.datetime.strptime(venta['creation_date'][0:19],'%Y-%m-%dT%H:%M:%S')
    limit = datetime.datetime.strptime('2020-08-27T00:12:24','%Y-%m-%dT%H:%M:%S')
    if(creation_date>limit):
        l_ventas3.append(venta)
        print(venta['creation_date'])

2020-08-31T10:03:26+02:00
2020-08-30T16:38:50+02:00


In [73]:
len(l_ventas3)

2

In [74]:
l_ventas3[0]['creation_date']

'2020-08-31T10:03:26+02:00'

In [48]:
len(l_ventas)

612

In [24]:
len(l_ventas)*2+432

1192

In [77]:
cuentas = {'paypal':'yfy4cPxkmFwIYVnoWD0A',
 'shopify_payments':'iA9Pzv2CImjItzwCaQv0'}

In [78]:
d_comisiones = {'paypal':0.019,
 'shopify_payments':0.021}
d_tasa = {'paypal':0.35,
 'shopify_payments':0.25}

In [79]:
i=1313
l_ventas2=[]
for venta in l_ventas3:
    d_venta = venta
    d_conta={
       "creation_date":"2020-08-26T18:02:55.591Z",
        "execution_date":d_venta['creation_date'][:-6]+'.000Z',
       "place":'spain',
        "project":'0DmODGTOEiM5lg9SGx0J',
        "intervention":'w04441aFcU5b7pQm6Rd2',
        "phase":None,
        # Dani
       "creator_user":"z5m936GA0t3vHM28QKhR",
        #Jose
        "user_in_charge":"IjBxXuBshlfq2MUzwBSK",
        "type":'income',
       "amount":d_venta['total_price'],
        #quitando los 4 € de envio
        "vat":round((d_venta['total_price']-4)*(1-1/1.21),2),
        # 3deqBmrZ1iJkAwMmxSOp - venta de manteca de karite
        "account_id":'3deqBmrZ1iJkAwMmxSOp',
        "target_id":cuentas[d_venta['gateway']],
       "concept":'venta shopify',
        "description":'venta online: '+str(d_venta['order_id']),
        "images":[],
        "code":i,
        "origin":'script_dani'
        
    }
    #calculo de comisiones
    comision=round(
                    (d_venta['total_price']*d_comisiones[d_venta['gateway']]
                    )+d_tasa[d_venta['gateway']],
                    2)
    d_comision={
       "creation_date":"2020-08-26T18:02:55.591Z",
        "execution_date":d_venta['creation_date'][:-6]+'.000Z',
       "place":'spain',
        "project":'0DmODGTOEiM5lg9SGx0J',
        "intervention":'w04441aFcU5b7pQm6Rd2',
        "phase":None,
        #dani
       "creator_user":"z5m936GA0t3vHM28QKhR",
        #jose
        "user_in_charge":"IjBxXuBshlfq2MUzwBSK",
        "type":'expense',
       "amount":comision,
        "vat":0,
        #j2UYEKf01eFHMPZv9qHq - gastos financieros
        "account_id":'j2UYEKf01eFHMPZv9qHq',
        "target_id":cuentas[d_venta['gateway']],
       "concept":'comision '+d_venta['gateway'],
        "description":'comision de venta: '+str(d_venta['order_id']),
        "images":[],
        "code":i+1,
        "origin":'script_dani'
        
    }
    i=i+2
    
    doc_acc = db.collection(u'Accounting').add(d_conta)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    doc_acc = db.collection(u'Accounting').add(d_comision)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    l_ventas2.append(d_conta)
    l_ventas2.append(d_comision)

In [81]:
#l_ventas2

In [54]:
len(l_ventas2)

4

In [30]:
#l_ventas2[len(l_ventas2)-1]

In [83]:
2*len(l_ventas)+432

1652

## VERIFICAR

In [ ]:
account_total = db.collection(
                       'Accounting'
                      ).where(u'origin', u'==', u'script_dani').stream()

# ELIMINAR

In [19]:
ventas_dani = db.collection('Accounting'
                           ).where(u'origin', u'==', u'script_dani').stream()

In [20]:
l_ventas = []
for venta in ventas_dani:
    l_ventas.append(venta.to_dict())

In [21]:
len(l_ventas)

0

In [18]:
l_ventas = []
for venta in ventas_dani:
    #l_ventas.append(venta.to_dict())
    d_venta=venta.to_dict()
    db.collection(u'Accounting').document(d_venta['id']).delete()

In [110]:
l_ventas = []
for venta in ventas_dani:
    l_ventas.append(venta.to_dict())

In [101]:
db.collection(u'Accounting').document('zwlvkHssBuygzyzGuoMA').delete()

seconds: 1598472978
nanos: 897772000

In [100]:
l_ventas[0]['id']

'06DwrZ0s8WZ7XsYAHFXj'

In [111]:
len(l_ventas)

0

# CREATE ACCOUNT

In [207]:
#Saca las cuentas 629
col_query = db.collection('Accounts').stream()

l_accounts=[]
for us in col_query:
    l_accounts.append(us.to_dict())
accnts = dict()
for us in l_accounts:
    if(str(us['number'])[0:3] == "629"):
        accnts.update({us['number']:us['name']})

In [210]:
cuenta={
    "db_creation_date": "2020-05-27T15:09:17.217220",
    "name":"Dietas (comida)",
    "number":629008,
    "transactions_id":[],
    "type":"expense",
    "user_id":None,
    "value":0
}

In [211]:
doc_acc = db.collection(u'Accounts').add(cuenta)[1]
doc_acc_id = doc_acc.id
doc_acc.update({'id':doc_acc_id})

update_time {
  seconds: 1590612517
  nanos: 206982000
}